# Create and Deploy an AI Service

This notebook provides a detailed demonstration of the steps and code required to showcase support for watsonx.ai AI service.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Goal
This notebook aims to demonstrate the application of Chat models, such as `mistralai/mistral-medium-2505`, using the tools provided by LangGraph. LangGraph serves as an Agent Orchestrator, enabling the development of graph-based applications that autonomously execute sequences of actions. In these applications, the Large Language Model (LLM) functions as the primary decision-maker, determining the subsequent steps. Following this, an AI service is created based on the previously constructed application. The AI Service is then deployed to a watsonx.ai Deployment Space. 

<b>Note:</b> This notebook provides the steps to create and deploy the market research agent, created in the previous lab exercise, as an AI service in watsonx.ai. This market research agent consists of various tools such as DuckDuckGo and Wikipedia. 



## Table of Contents (UPDATE) 
 
This notebook contains the following parts:

- [Setup](#setup)
- [Create AI service](#ai_service)
- [Testing AI service's function locally](#testing)
- [Deploy AI service](#deploy)
- [Example of Executing an AI service](#example)
- [Summary](#summary)

## Environment Setup


### Watsonx.ai Runtime Service
Before you use the sample code in this notebook, ensure an instance of the watsonx.ai Runtime Service is created and associated to the project. The lab guide provides more information on how to associate the watsonx.ai Runtime Service to the watsonx.ai project.

### Install and import the `datasets` and dependencies

In [1]:
!pip install -U "langgraph>0.2,<0.3" | tail -n 1
!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1
!pip install ddgs
!pip install wikipedia

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

Enter your watsonx.ai api key and hit enter:  ········


### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below


In [3]:
import os

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your space_id (hit enter): ")

Please enter your space_id (hit enter):  f9e90bd4-d87e-4a51-b53a-ba417f68d1c9


Create an instance of APIClient with authentication details.

In [4]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, space_id=space_id)

Specify the `model_id` of the model you will use for the chat with tools.

In [5]:
model_id = "mistralai/mistral-medium-2505"

<a id="ai_service"></a>
## Create AI service

The content of this notebook is derived from and based on the material presented in the [Use watsonx, and `mistralai/mistral-large` with support for tools to perform simple calculations](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/deployments/foundation_models/chat/Use%20watsonx%2C%20and%20%60mistral-large%60%20with%20support%20for%20tools%20to%20perform%20simple%20calculations.ipynb) notebook. 

Prepare function which will be deployed using AI service.

In [6]:
def deployable_ai_service(context, url=credentials["url"], model_id=model_id, **kwargs):
    
    from ibm_watsonx_ai import APIClient, Credentials
    from langchain_ibm import ChatWatsonx
    from langchain_core.tools import tool
    from langgraph.prebuilt import create_react_agent

    # Needed for DuckDuckGo search tool
    import ddgs 
    
    from ddgs import DDGS
    
    ## Needed for Wikipedia Tool
    from langchain.tools import WikipediaQueryRun
    from langchain.utilities import WikipediaAPIWrapper
    
    from langchain.chat_models import ChatOpenAI # remove this?

    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id=context.get_space_id(),
    )

    chat = ChatWatsonx(
        watsonx_client=api_client,
        model_id=model_id,
        params={"temperature": 0.1}
    )
    
    
    @tool
    def duckduckgo_search(query: str) -> str:
        """Use this tool to search the web using DuckDuckGo. Input should be a search query."""    
        with DDGS() as ddgs:
            results = ddgs.text(query)
            if results:
                return results[0]['body']  # Return the snippet of the first result
            else:
                return "No results found."
    
    @tool
    def wikipedia_tool(query: str) -> str:
        """Useful for answering questions about general knowledge or historical topics using Wikipedia."""    
        wiki = WikipediaAPIWrapper()
        func=WikipediaQueryRun(api_wrapper=wiki).run
    
    
    tools = [duckduckgo_search, wikipedia_tool]
    
    graph = create_react_agent(chat, tools=tools)

    def generate(context) -> dict:
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        question = payload["question"]
        
        response = graph.invoke({"messages": [("user", f"{question}")]})
        
        json_messages = [msg.to_json() for msg in response["messages"]]
        
        response["messages"] = json_messages
        
        return {"body": response}
    
    def generate_stream(context):
        api_client.set_token(context.get_token())
   
        payload = context.get_json()
        question = payload["question"]
        
        for el in graph.stream({"messages": [("user", f"{question}")]}, stream_mode="values"):
            json_messages = [msg.to_json() for msg in el["messages"]]
            el["messages"] = json_messages
            yield el
        
    return generate, generate_stream

Add a helpful function to print messages from the model.

In [7]:
import json

def print_message(message):
    last_message_id = message["id"][-1]
    print(f" ===== {last_message_id} =====")

    match last_message_id:
        case "AIMessage":
            content = message["kwargs"].get("additional_kwargs", message["kwargs"].get("content"))
            print(
                json.dumps(content, indent=2)
                if isinstance(content, dict)
                else content,
                end="\n\n"
            )
        case "ToolMessage":
            print(message["kwargs"]["name"])
            print(message["kwargs"]["content"], end="\n\n")
        case _:
            print(message["kwargs"]["content"], end="\n\n")


def ai_services_pretty_print(iter):
    if "body" in iter:
        iter = iter["body"]

    for message in iter["messages"]:
        print_message(message)


def ai_services_pretty_print_stream(iter):
    for el in iter:
        try:
            el = json.loads(el)
        except TypeError:
            pass

        print_message(el["messages"][-1])

<a id="testing"></a>
## Testing AI service's function locally

You can test AI service's function locally. Initialise `RuntimeContext` firstly.

In [8]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=api_client)

In [9]:
local_function = deployable_ai_service(context=context)

Preapre request json payload.

In [10]:
context.request_payload_json = {"question": "What are the latest news headlines about IBM?"}

Execute the `generate` function locally.

In [12]:
resp = local_function[0](context)

In [13]:
ai_services_pretty_print(resp)

 ===== HumanMessage =====
What are the latest news headlines about IBM?

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "EWrwxuXJM",
      "type": "function",
      "function": {
        "name": "duckduckgo_search",
        "arguments": "{\"query\": \"latest news headlines about IBM\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
duckduckgo_search
Get the latest IBM news .What IBM does. The mission of IBM is to be a catalyst that makes the world work better. We aim to have a positive impact globally, and in the communities where we operate, through business ethics, environmental commitment and responsible technology.

 ===== AIMessage =====
Here are some of the latest news headlines about IBM:

1. **IBM and AWS Expand Partnership to Accelerate Cloud Adoption**: IBM and Amazon Web Services (AWS) have expanded their partnership to help enterprises accelerate their cloud adoption journey. The collaboration aims to provide customers with more flexibility and choice in their

In [14]:
response = local_function[1](context)

In [15]:
ai_services_pretty_print_stream(response)

 ===== HumanMessage =====
What are the latest news headlines about IBM?

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "REscug9Oq",
      "type": "function",
      "function": {
        "name": "duckduckgo_search",
        "arguments": "{\"query\": \"latest news headlines about IBM\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
duckduckgo_search
Further details as to the scope of Adept remain at large, yet the concept is just the latest in the slew of similar news to have come from IBM in the last 12 months.

 ===== AIMessage =====
Here are some of the latest news headlines about IBM:

1. IBM has been involved in several initiatives over the past year, including advancements in AI and other technologies.
2. Specific details about a project named "Adept" are still emerging, but it is part of a series of recent developments from IBM.

For more detailed and specific headlines, you might want to check the latest news directly from a reliable news source or IBM's official 

<a id="deploy"></a>
## Deploy AI service

Prepare a configuration file for defining a custom software specification.

In [16]:
config_yml = """
name: python311
channels:
  - empty
dependencies:
  - pip:
    - langgraph==0.2.44
    - ddgs
    - wikipedia
    - langchain
prefix: /opt/anaconda3/envs/python311
"""

with open("config.yaml", "w", encoding="utf-8") as f:
    f.write(config_yml)

In [17]:
base_sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
meta_prop_pkg_extn = {
    api_client.package_extensions.ConfigurationMetaNames.NAME: "watsonx.ai env with langgraph",
    api_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Environment with langgraph",
    api_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml"
}

pkg_extn_details = api_client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="config.yaml")
pkg_extn_id = api_client.package_extensions.get_id(pkg_extn_details)
pkg_extn_id

Creating package extensions
SUCCESS


'e1ba4fc2-2d93-427d-b15b-e0fe1fe643e5'

In [18]:
meta_prop_sw_spec = {
    api_client.software_specifications.ConfigurationMetaNames.NAME: "AI service watsonx.ai custom software specification with langgraph",
    api_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for AI service deployment",
    api_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_id}
}

sw_spec_details = api_client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_id = api_client.software_specifications.get_id(sw_spec_details)
api_client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_id)
sw_spec_id

SUCCESS


'5a13044e-55af-4de8-8756-be5248936e02'

In [19]:
meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: "AI service SDK with langgraph",    
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(deployable_ai_service, meta_props)

In [20]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

'5d6ecfd7-63aa-4244-8753-67bb30353f77'

In [21]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: "AI service with tools",
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)



######################################################################################

Synchronous deployment creation for id: '5d6ecfd7-63aa-4244-8753-67bb30353f77' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
..............
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='b00bd6c7-7d4e-4323-8362-4bbe787a5ec5'
-----------------------------------------------------------------------------------------------




Obtain the `deployment_id` of the previously created deployment.

In [22]:
deployment_id = api_client.deployments.get_id(deployment_details)

<a id="example"></a>
## Example of Executing an AI service.

Execute `generate` method.

In [23]:
question = "What are the latest news headlines about IBM?"

deployments_results = api_client.deployments.run_ai_service(
    deployment_id, {"question": question}
)

In [24]:
ai_services_pretty_print(deployments_results)

 ===== HumanMessage =====
What are the latest news headlines about IBM?

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "function": {
        "arguments": "{\"query\": \"latest news headlines about IBM\"}",
        "name": "duckduckgo_search"
      },
      "id": "m2T3RVTjP",
      "type": "function"
    }
  ]
}

 ===== ToolMessage =====
duckduckgo_search
By Justina Nixon-Saintil | VP, CSR and Chief Impact Officer NORTHAMPTON, MA / ACCESS Newswire / March 6, 2025 / IBM has been working with ...

 ===== AIMessage =====
Here are some of the latest news headlines about IBM:

1. **IBM's Strategic Moves in AI and Cloud Computing**:
   - IBM has been making significant strides in AI and cloud computing, focusing on enhancing its hybrid cloud and AI capabilities to drive innovation and efficiency across various industries.

2. **IBM's Commitment to Sustainability**:
   - IBM has reaffirmed its commitment to sustainability, with initiatives aimed at reducing carbon emissions and promot

In [25]:
question = "What are the latest news headlines about Agentic AI?"

deployments_results = api_client.deployments.run_ai_service_stream(
    deployment_id, {"question": question}
)

In [26]:
ai_services_pretty_print_stream(deployments_results)

 ===== HumanMessage =====
What are the latest news headlines about Agentic AI?

 ===== AIMessage =====
{
  "tool_calls": [
    {
      "id": "Ge1a727D3",
      "type": "function",
      "function": {
        "name": "duckduckgo_search",
        "arguments": "{\"query\": \"latest news headlines about Agentic AI\"}"
      }
    }
  ]
}

 ===== ToolMessage =====
duckduckgo_search
Over 40% of agentic AI projects will be canceled by the end of 2027 , due to escalating costs, unclear business value or inadequate risk controls, according to Gartner.

 ===== AIMessage =====
Here is the latest news headline about Agentic AI:

- Over 40% of agentic AI projects will be canceled by the end of 2027, due to escalating costs, unclear business value, or inadequate risk controls, according to Gartner.



<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to create and deploy an AI service using the `ibm_watsonx_ai` SDK.

Check out the _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

This notebook was modified from the original notebook created by Mateusz Szewczyk, Software Engineer at watsonx.ai. View the <a href ="https://github.com/IBM/watsonx-ai-samples/blob/master/cloud/notebooks/python_sdk/deployments/foundation_models/chat/Use%20watsonx%2C%20and%20%60llama-3-3-70b-instruct%60%20to%20create%20ai%20service.ipynb"> original notebook here</a>.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.